In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from scipy.ndimage import gaussian_filter
from sklearn.preprocessing import OneHotEncoder
import random
from keras.utils.np_utils import to_categorical   
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import class_weight

In [3]:
train_list_x = os.listdir("../input/mlsp-train2/X")

In [4]:
x_train = []
y_train = []
for file in train_list_x :
    x_train.append(np.load(os.path.join("../input/mlsp-train2/X" , file) ,allow_pickle = True))
    y_train.append(np.load(os.path.join("../input/mlsp-train2/Y" , "eventroll"+ file[7:]) ,allow_pickle = True))
x_train= np.array(x_train)
y_train = np.array(y_train)

In [5]:
np.save("./x_train.npy" , x_train)
np.save("./y_train.npy" , y_train)

In [18]:
x_train = np.load("./x_train.npy")
y_train = np.load("./y_train.npy")

In [7]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [30]:
y = []
for i in y_train :
    y.append(eventroll_to_multihot_vector(i))

In [31]:
y_train = np.array(y)

In [19]:
def addDim(mat) :
  return np.expand_dims(mat , axis = 2)
def reduceDim(mat) :
  return mat[0]

In [21]:
x = []
for x1 in x_train :
    x.append(addDim(reduceDim(x1)))
x_train = np.array(x)

In [32]:
np.shape(y_train)

(10000, 10)

In [22]:
np.shape(x_train)

(10000, 64, 1000, 1)

In [24]:
def cnn() :
    cnn = Sequential()
    cnn.add(layers.Conv2D(16,(3,3),(2,2), padding = 'same' , activation = 'relu' ,input_shape = (64,1000,1,)))
    cnn.add(layers.Conv2D(16,(3,3),(2,2), padding = 'same' , activation = 'relu' ))
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(32 , (3,3) ,padding = 'same' , activation = 'relu'))
    cnn.add(layers.Conv2D(32 , (3,3) ,padding = 'same' , activation = 'relu'))
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) ,(2,2), padding = 'same' , activation = 'relu'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) , padding = 'same'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.GlobalAveragePooling2D())
    cnn.add(layers.Dense(64 , activation = 'relu'))
    cnn.add(layers.Dense(10 , activation = 'sigmoid'))
    cnn.summary()
    return cnn

In [25]:
model = cnn()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 500, 16)       160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 250, 16)       2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 125, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 125, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 125, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 125, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 62, 32)         0

2022-11-14 07:49:22.428378: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [34]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss= 'binary_crossentropy' , metrics=['binary_accuracy'])

In [35]:
history = model.fit(
    x = x_train , y = y_train,
    validation_split = 0.2,
    epochs = 80 , 
    steps_per_epoch = 100 ,
    batch_size = 32,
    
)

Epoch 1/80
100/100 [==============================] - 12s 110ms/step - loss: 0.4014 - binary_accuracy: 0.8486 - val_loss: 0.4145 - val_binary_accuracy: 0.8719
Epoch 2/80
100/100 [==============================] - 11s 111ms/step - loss: 0.3178 - binary_accuracy: 0.8759 - val_loss: 0.4822 - val_binary_accuracy: 0.8719
Epoch 3/80
100/100 [==============================] - 11s 106ms/step - loss: 0.2999 - binary_accuracy: 0.8832 - val_loss: 0.4849 - val_binary_accuracy: 0.8723
Epoch 4/80
100/100 [==============================] - 10s 103ms/step - loss: 0.2772 - binary_accuracy: 0.8900 - val_loss: 0.4599 - val_binary_accuracy: 0.8330
Epoch 5/80
100/100 [==============================] - 11s 110ms/step - loss: 0.2803 - binary_accuracy: 0.8890 - val_loss: 0.4586 - val_binary_accuracy: 0.8470
Epoch 6/80
100/100 [==============================] - 10s 102ms/step - loss: 0.2670 - binary_accuracy: 0.8944 - val_loss: 0.3787 - val_binary_accuracy: 0.8558
Epoch 7/80
100/100 [==========================

In [36]:
model.save('./multi2.h5')

In [37]:
model = keras.models.load_model('./multi2.h5')

In [38]:
model = keras.models.load_model("./multi2.h5")
dict = {}
test_list = os.listdir("../input/mlsp-test/test/X")
for file in test_list:
    a = np.load(os.path.join("../input/mlsp-test/test/X", file))
    a =  np.expand_dims(a, axis = 3)
    dict[file] = model.predict(a)
pred = np.array(list(dict.values()))
pred = np.where(pred>=0.5 , 1, 0)

In [40]:
np.shape(pred)

(2500, 1, 10)

In [41]:
results = []
for i in pred :
    a = []
    for j in range(10) :
        a.append(i[0][j])
    results.append(a)

In [42]:
field_ids = list(dict.keys())

In [43]:
names = {
    
    0: 'Alarm_bell_ringing', 
    1: 'Blender', 
    2: 'Cat', 
    3: 'Dishes', 
    4: 'Dog',
    5: 'Electric_shaver_toothbrush', 
    6: 'Frying', 
    7: 'Running_water',
    8: 'Speech', 
    9: 'Vacuum_cleaner'


}

In [44]:
arr = []
for t in results :
    vec = []
    for i in range(10) :
        if t[i]==1:
            vec.append(names[i])
    arr.append(vec)

In [46]:
df = pd.DataFrame(list(zip(field_ids, arr)),columns =['fileid', 'prediction'])

In [47]:
df.to_csv("./200046_pred.csv")